# ID3,C4.5和CART算法python实现

### 信息增益

决策树中的ID3和c4.5算法是基于香农信息论。
香农熵：给定可能性分布P和实例S， 那么信息熵可以如下计算：  
$Entropie(P) =-\sum_{i=1}^{n}p_i * log(p_i)$

信息增益G(P, T): 决策树通过对不同特征进行切分判断来进行分类。那怎么判断什么哪个特征比较合适。可以用信息增益来测量所有样本特征的混合程度，选择比较混乱的特征(熵比较大), 经过叫少的判断即可区分。计算公式如下：  
$Gain(p, T)  = Entropie(P) - \sum_{i=1}^{n}(p_i * Entropie(p_i))$  
pi表示T特征中发生的可能性。

下面通过一个例子来进行简单计算：

In [1]:
import numpy as np
import pandas as pd
from math import log

#加载数据
df = pd.read_csv('./example_data.csv')
df

,humility,outlook,play,temp,windy
0,high,sunny,no,hot,False
1,high,sunny,no,hot,True
2,high,overcast,yes,hot,False
3,high,rainy,yes,mild,False
4,normal,rainy,yes,cool,False
5,normal,rainy,no,cool,True
6,normal,overcast,yes,cool,True
7,high,sunny,no,mild,False
8,normal,sunny,yes,cool,False
9,normal,rainy,yes,mild,False


In [2]:
# 定义简单的熵计算函数：
def entropy(label_set):
    return -np.sum(label_set * np.log2(label_set))

In [3]:
# 全集S的熵：
entropy_s = entropy([9/14, 5/14]) # 两个类别
entropy_s

0.9402859586706311

In [4]:
for col in ['outlook', 'temp', 'humility', 'windy']:
    print(df.groupby([col, 'play']).size())
    print('-' * 30)

outlook   play
overcast  yes     4
rainy     no      2
          yes     3
sunny     no      3
          yes     2
dtype: int64
------------------------------
temp  play
cool  no      1
      yes     3
hot   no      2
      yes     2
mild  no      2
      yes     4
dtype: int64
------------------------------
humility  play
high      no      4
          yes     3
normal    no      1
          yes     6
dtype: int64
------------------------------
windy  play
False  no      2
       yes     6
True   no      3
       yes     3
dtype: int64
------------------------------


In [5]:
# 其中每个类别对应的play情况如上， 因此信息增益如下计算：
gain_s_outlook = entropy_s - 4/14 * entropy([1]) - 5/14*entropy([2/5, 3/5]) - 5/14*entropy([3/5, 2/5])
# 同样方法可以计算出另外两个特征的信息增益
gain_s_temperature = entropy_s - 4/14*entropy([1/4, 3/4]) - 4/14 * entropy([2/4, 2/4]) - 6/14*entropy([2/6, 4/6])
gain_s_humidity = entropy_s - 7/14 * entropy([4/7, 3/7]) - 7/14 * entropy([1/7, 6/7])
gain_s_wind = entropy_s - 5/14*entropy([3/5, 2/5]) - 9/14 * entropy([2/9, 7/9])

In [6]:
# 4个特征的信息增益计算完毕，选组最大的特征作为根节点进行切分
gain_s_outlook, gain_s_humidity, gain_s_wind, gain_s_temperature

(0.24674981977443933,
 0.15183550136234164,
 0.10224356360985076,
 0.02922256565895487)

选择outlook作为根节点切分之后，分为3个分支。其中overcast分支不用再进行判断。其余两个分支按照同样方法进行切分，直到不可再分。
这就是ID3算法的计算过程。
缺点：如上计算，假设outlook中共计有14个类别， 其信息增益当然最大， 应该选择作为根节点切分，但此时并不一定是最好的切分特征，因为不具有泛化能力。此时选择信息增益比来进行改进。

### 信息增益比

可以使用如下公式计算：  
先计算splitinfo：  
$splitinfo(p, T) = -\sum_{j=1}^{n}(\frac{|p_j|}{p}) * log2(\frac{|p_j|}{p})$ 

再计算gainratio：  
$gainratio(p, T) = \frac{Gain(P, T)}{splitinfo(p, T)}$  

上面的过程已经计算除了分子，下面主要是分母的计算，如下：

In [7]:
splitinfo_s_outlook = entropy([5/14, 5/14, 4/14])
splitinfo_s_temperature = entropy([4/14, 4/ 14, 6/14])
splitinfo_s_humidity = entropy([7/14, 7/14])
splitinfo_s_wind = entropy([5/14, 9/14])

In [8]:
print(splitinfo_s_outlook)
print(splitinfo_s_temperature)
print(splitinfo_s_humidity)
print(splitinfo_s_wind)

1.5774062828523454
1.5566567074628228
1.0
0.9402859586706311


In [9]:
print(gain_s_outlook / splitinfo_s_outlook)
print(gain_s_temperature / splitinfo_s_temperature)
print(gain_s_humidity / splitinfo_s_humidity)
print(gain_s_wind / splitinfo_s_wind)

0.15642756242117528
0.018772646222418813
0.15183550136234164
0.1087366695918781


选择信息增益比高的特征进行类别切分，其后过程和ID3算法类似。
经过以上计算，上述两种决策树还有已下特点：
1. 缺失值的处理
如果某些特征有少量缺失值，决策树通过计算信息增益比，比如上述outlook特征中某个值缺失，但计算的结果相差不大， 因此可以处理少量缺失值的数据。
2. 连续行特征
与ID3只能处理类别数据不同，C4.5还可以处理连续数据。方法是去连续数据中间的值，将该特征分为两类，进而计算信息增益比，也是选则最大值进行切分。
3. 剪枝
根据训练集生成决策树，通常会对数据过拟合，并且对一些噪声值比较敏感。为了提高正确率， 需要对树进行剪枝。剪枝是一种通过删除树中提供很少分类实例的部分来减少决策树的大小的一种方法。

### 基尼指数（Gini index）

可以使用如下公式计算：  
先计算splitinfo：

$Gini(D) = \sum_{k=1}^{K}{p_k}({1-p_k})=1-\sum_{k=1}^{K}{p_k^2}$ 

再计算gainratio：  
$Gini(D|A) = \sum_{k=1}^{K}\frac{|D_k|}{|D|}{Gain(D_k)}$  

上面的过程已经计算除了分子，下面主要是分母的计算，如下：

In [10]:
#输入数值型参数
#定义Gini基尼指数函数
def gini(nums):
#     return 1 - np.sum(np.power(nums, 2))
    gini = np.sum([p*(1-p) for p in nums]) 
    return gini 

In [11]:
# 可以看到不同的特征分为两类， 计算基尼系数，选择较小的值进行切分:
gini_s_outlook = gini([5/14, 4/14,5/14])
gini_s_temperature = gini([4/14, 4/14, 6/14])
gini_s_humidity = gini([7/14, 7/14])
gini_s_wind = gini([8/14, 6/14])

In [12]:
gini_s_outlook, gini_s_humidity, gini_s_wind, gini_s_temperature

(0.6632653061224489, 0.5, 0.4897959183673469, 0.6530612244897959)


因此选择最小的特征wind，构建利用上述分为两部分分别计算gini系数，从而确定分类类别，构建二叉树进行决策。